# Format Climate Features

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-01

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Format Climate Features
# Author: Timm Nawrocki
# Created on: 2019-04-01
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Format Climate Features" calculates two-decadal averages for Date of Thaw, Date of Freeze, Growing Season Length, Summer Warmth Index, and Annual Precipitation.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')
climate_source = os.path.join(root_directory, 'Data_Input/source_data/climatology')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
area_of_interest = os.path.join(root_directory, 'Data_Input/area_of_interest/area_of_interest.tif')
dateOfFreeze_2000s = os.path.join(climate_source, 'Alaska_DateOfFreeze_2000s.tif')
dateOfFreeze_2010s = os.path.join(climate_source, 'Alaska_DateOfFreeze_2010s_RCP60.tif')
dateOfThaw_2000s = os.path.join(climate_source, 'Alaska_DateOfThaw_2000s.tif')
dateOfThaw_2010s = os.path.join(climate_source, 'Alaska_DateOfThaw_2010s_RCP60.tif')
growingSeasonLength_2000s = os.path.join(climate_source, 'Alaska_GrowingSeasonLength_2000s.tif')
growingSeasonLength_2010s = os.path.join(climate_source, 'Alaska_GrowingSeasonLength_2010s_RCP60.tif')
annualPrecipitation_2000s = os.path.join(climate_source, 'Alaska_PrecipitationAnnual_2000s.tif')
annualPrecipitation_2010s = os.path.join(climate_source, 'Alaska_PrecipitationAnnual_2010s_RCP60.tif')
tempMay_2000s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_cru_TS31_historical_05_2000_2009.tif')
tempMay_2010s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_5modelAvg_rcp60_05_2010_2019.tif')
tempJune_2000s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_cru_TS31_historical_06_2000_2009.tif')
tempJune_2010s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_5modelAvg_rcp60_06_2010_2019.tif')
tempJuly_2000s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_cru_TS31_historical_07_2000_2009.tif')
tempJuly_2010s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_5modelAvg_rcp60_07_2010_2019.tif')
tempAugust_2000s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_cru_TS31_historical_08_2000_2009.tif')
tempAugust_2010s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_5modelAvg_rcp60_08_2010_2019.tif')
tempSeptember_2000s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_cru_TS31_historical_09_2000_2009.tif')
tempSeptember_2010s = os.path.join(climate_source, 'tas_decadal_mean_monthly_mean_c_5modelAvg_rcp60_09_2010_2019.tif')

# Define output names
output_names = ['dateFreeze.tif',
                'dateThaw.tif',
                'growingSeason.tif',
                'precipitation.tif',
                'summerWarmth.tif'
               ]

# Define output path
output_path = os.path.join(root_directory, 'Data_Input/predictor_env')

# Define output files
output_array = []
for name in output_names:
    file = os.path.join(output_path, name)
    output_array = output_array + [file]

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 1. Define Functions

In [4]:
# Define a function to calculate decadal summer warmth index
def summer_warmth(**kwargs):
    """
    Description: calculates the sum of degrees per month across the months of May, June, July, August, and September to represent overall summer temperature
    Inputs: 'input_array' -- an array containing the input rasters for each of five summer decadal mean monthly temperatures ordered from May to September
            'output_array' -- an array of containing the location to save the output decadal mean summer warmth index
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_rasters = kwargs['input_array']
    output_raster = kwargs['output_array'][0]
    # Set the snap raster and cell size environments to match the climate rasters
    arcpy.env.snapRaster = input_rasters[0]
    arcpy.env.cellSize = input_rasters[0]
    # Calculate summer warmth index and copy results to output raster
    print('Calculating summer warmth index...')
    summer_warmth_raster = ((Raster(input_rasters[0]) * 31)
                            + (Raster(input_rasters[1]) * 30)
                            + (Raster(input_rasters[2]) * 31)
                            + (Raster(input_rasters[3]) * 31)
                            + (Raster(input_rasters[4]) * 30))
    print('Copying results to output raster...')
    arcpy.CopyRaster_management(summer_warmth_raster,
                                output_raster,
                                '',
                                '',
                                '-32768',
                                'NONE',
                                'NONE',
                                '16_BIT_SIGNED',
                                'NONE',
                                'NONE',
                                'TIFF',
                                'NONE'
                               )
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully calculated summer warmth index...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define a function to calculate the average of multiple decades for climate metrics
def average_climate(**kwargs):
    """
    Description: extracts topographic rasters to the area of interest raster with grid and cell size matched to the study area raster
    Inputs: 'input_array' -- an array containing the input rasters with the first raster as the area of interest and any number of additional climate rasters representing different decades for the same metric
            'output_array' -- an array of one output name per input raster
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_rasters = kwargs['input_array']
    area_of_interest = input_rasters.pop(0)
    output_raster = kwargs['output_array'][0]
    # Set the snap raster and cell size environments to match the climate rasters
    arcpy.env.snapRaster = input_rasters[0]
    arcpy.env.cellSize = input_rasters[0]
    # Set the raster sum initially equal to the first raster
    print('Summing input rasters...')
    raster_sum = Raster(input_rasters[0])
    # Add the values of additional rasters in the list
    for raster in input_rasters[1:]:
        raster_sum += Raster(raster)
    # Find the mean by dividing by the total number of rasters and copy results to output raster
    print('Calculating mean of summed rasters...')
    n = len(input_rasters)
    raster_mean = raster_sum / n
    raster_int = Int(RoundDown(raster_mean + 0.5))
    # Set the snap raster and cell size environments to match the area of interest
    arcpy.env.snapRaster = area_of_interest
    arcpy.env.cellSize = area_of_interest
    # Extract to area of interest
    print('Extracting raster to area of interest...')
    raster_extract = ExtractByMask(raster_int, Raster(area_of_interest))
    # Copy results to output raster
    print('Copying results to output raster...')
    arcpy.CopyRaster_management(raster_extract,
                                output_raster,
                                '',
                                '',
                                '-32768',
                                'NONE',
                                'NONE',
                                '16_BIT_SIGNED',
                                'NONE',
                                'NONE',
                                'TIFF',
                                'NONE'
                               )
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully calculated climate multi-decadal average...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

## 3. Calculate Date of Freeze

In [ ]:
# Define input and output arrays for date of freeze (dof)
dof_inputs = [area_of_interest,
              dateOfFreeze_2000s,
              dateOfFreeze_2010s
             ]
dof_outputs = [output_array[0]]

# Create key word arguments
dof_kwargs = {'input_array' : dof_inputs,
              'output_array' : dof_outputs
             }

# Calculate the mean date of freeze for 2000-2019
arcpy_geoprocessing(average_climate, **dof_kwargs)

## 4. Calculate Date of Thaw

In [ ]:
# Define input and output arrays for date of thaw (dot)
dot_inputs = [area_of_interest,
              dateOfThaw_2000s,
              dateOfThaw_2010s
             ]
dot_outputs = [output_array[1]]

# Create key word arguments
dot_kwargs = {'input_array' : dot_inputs,
              'output_array' : dot_outputs
             }

# Calculate the mean date of thaw for 2000-2019
arcpy_geoprocessing(average_climate, **dot_kwargs)

## 5. Calculate Growing Season Length

In [ ]:
# Define input and output arrays for growing season length (gsl)
gsl_inputs = [area_of_interest,
              growingSeasonLength_2000s,
              growingSeasonLength_2010s
             ]
gsl_outputs = [output_array[2]]

# Create key word arguments
gsl_kwargs = {'input_array' : gsl_inputs,
              'output_array' : gsl_outputs
             }

# Calculate the mean growing season length for 2000-2019
arcpy_geoprocessing(average_climate, **gsl_kwargs)

## 6. Calculate Annual Precipitation

In [ ]:
# Define input and output arrays for annual precipitation
precipitation_inputs = [area_of_interest,
                        annualPrecipitation_2000s,
                        annualPrecipitation_2010s
                       ]
precipitation_outputs = [output_array[3]]

# Create key word arguments
precipitation_kwargs = {'input_array' : precipitation_inputs,
                        'output_array' : precipitation_outputs
                       }

# Calculate the total annual precipitation for 2000-2019
arcpy_geoprocessing(average_climate, **precipitation_kwargs)

## 7. Calculate Summer Warmth Index

In [6]:
# Define output for summer warmth index 2000s
summerWarmth_2000s = os.path.join(climate_source, 'Alaska_SummerWarmthIndex_2000s.tif')

# Define input and output arrays for summer warmth index 2000s
summer_2000s_inputs = [area_of_interest,
                        tempMay_2000s,
                        tempJune_2000s,
                        tempJuly_2000s,
                        tempAugust_2000s,
                        tempSeptember_2000s
                       ]
summer_2000s_outputs = [summerWarmth_2000s]

# Create key word arguments
summer_2000s_kwargs = {'input_array' : summer_2000s_inputs,
                        'output_array' : summer_2000s_outputs
                       }

# Calculate the 2000s summer warmth index
arcpy_geoprocessing(summer_warmth, **summer_2000s_kwargs)

Calculating summer warmth index...
Copying results to output raster...
Successfully calculated summer warmth index...
----------
Succeeded at 2019-04-02 18:02 (Elapsed time: 0:07:27)


In [7]:
# Define output for summer warmth index 2010s
summerWarmth_2010s = os.path.join(climate_source, 'Alaska_SummerWarmthIndex_2010s_RCP60.tif')

# Define input and output arrays for summer warmth index 2010s
summer_2010s_inputs = [area_of_interest,
                        tempMay_2010s,
                        tempJune_2010s,
                        tempJuly_2010s,
                        tempAugust_2010s,
                        tempSeptember_2010s
                       ]
summer_2010s_outputs = [summerWarmth_2010s]

# Create key word arguments
summer_2010s_kwargs = {'input_array' : summer_2010s_inputs,
                        'output_array' : summer_2010s_outputs
                       }

# Calculate the 2010s summer warmth index
arcpy_geoprocessing(summer_warmth, **summer_2010s_kwargs)

Calculating summer warmth index...
Copying results to output raster...
Successfully calculated summer warmth index...
----------
Succeeded at 2019-04-02 18:10 (Elapsed time: 0:07:31)


In [8]:
# Define input and output arrays for mean summer warmth index
summer_mean_inputs = [area_of_interest,
                      summerWarmth_2000s,
                      summerWarmth_2010s
                     ]
summer_mean_outputs = [output_array[4]]

# Create key word arguments
summer_mean_kwargs = {'input_array' : summer_mean_inputs,
                      'output_array' : summer_mean_outputs
                     }

# Calculate the summer warmth index for 2000-2019
arcpy_geoprocessing(average_climate, **summer_mean_kwargs)

F:/ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS\Data_Input/predictor_env\summerWarmth.tif already exists and will be overwritten.
Summing input rasters...
Calculating mean of summed rasters...
Extracting raster to area of interest...
Copying results to output raster...
Successfully calculated climate multi-decadal average...
----------
Succeeded at 2019-04-02 18:20 (Elapsed time: 0:10:41)
